### 자연어 감성 분석
- 감성사전 기반 : 미리 저으이된 감성단어 사전 사용(규칙 기반)
    - TextBlob, AFINNm VADER
- 머신러닝 기반 : 데이터로 부터 패턴 학습(통계 기반)
    - TF-IDF 벡터화
    - 빈도수 벡터화
    - Multinomial Naive Bayes
    - 로지스틱 회귀

### 사용 데이터
- NLTK 영화 리뷰 (2000개)
- 다음영화리뷰

#### 알고리즘
- textblob
- afinn
- vader

- TF-IDF
- CounterVectorizer
- Multinomial Naive Bayes
- 로지스틱 회귀 ^

### TextBlob

In [ ]:
# TextBlob
# 특정단어를 선정하고, 그 단어가 나올경우 점수를 추가하거나 감점한다.
# Polarity  극성도
# 긍정(1)과 부정(-1)
# 0은 중립
# Subjectivity 주관성
# 객관(-1)과 주관(1)
# 0은 중립

# 문맥은 무시하고 단어 여부로만 판단한다.
# 이 영화는 나쁘지 않다. -> 나쁘다, 않다 에서 감점(-)으로 인식
# 장점 : 빠른속도 학습 불필요
# 사용 : 실시간 감성 분석, 스트리밍 데이터 -> 빠른 판단이 필요한 상황에 사용하기도 한다.

In [ ]:
# %pip install TextBlob

In [ ]:
from textblob import TextBlob, Word
text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob = TextBlob(text)
print(blob.sentences)
print(blob.words)
print(blob.tags)

import nltk
# nltk.download('brown')
print(blob.noun_phrases)

# 문장, 단어, 품사 단위의 기본 분석 기능을 지원함을 확인할 수 있다.
# 단어 분리의 경우에는 무엇을 기준으로? ^ 공백, 문장부호, 형태소 분석 규칙을 기준

In [ ]:
# 감성 분석
blob.sentiment
# Sentiment(polarity=0.5, subjectivity=0.6785714285714286)

# Polarity  극성도
# 긍정(1)과 부정(-1)
# 0은 중립
# Subjectivity 주관성
# 객관(-1)과 주관(1)
# 0은 중립

# 약간 긍정이고, 주관적인 표현이다.

print(f'polarity : {blob.sentiment.polarity}')
print(f'subjectivity : {blob.sentiment.subjectivity}')

### AFINN

In [ ]:
# AFINN(Lexicon-Based)
# 각 단어의 -5 ~ +5의 점수를 부여하고 합산
# 이 영화는 좋지만 좋지않은 부분도 있다.
    # 좋다 +3 좋다 +3 않다 -3 = +3 > 0 -> 긍정 으로 평가
# 조회 방법 : score = sum(word_sentiment_value)
# 분류규칙
    # 0 > score 긍정
    # score < 0 부정
# 이모티콘지원
# 강도표현 인식 very, really 등

# 강조 수정자(intensifiers)
    # 매우좋다 = 1.5 x (좋다의 점수)

# AFINN vs TextBlob
# AFINN : 더 정확한 점수 매핑
# TextBlob : 더 일반적인 접근 ^

In [8]:
# %pip install afinn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53479 sha256=d5e78352c9a512aa4d0d73e7468f036fd2534ea239a3df9b2abd21038a65bdfb
  Stored in directory: c:\users\playdata2\appdata\local\pip\cache\wheels\f3\20\88\bd79bf1dfa529d0e4e301372371edf7639514a9f0d337769c3
Successfully built afinn
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'afinn' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'afinn'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
from afinn import Afinn
af = Afinn()
text1 = "TextBlob is amazingly simple to use."
text2 = "What a wonderful library for NLP!"
score1 = af.score(text1)
score2 = af.score(text2)
score1, score2

(0.0, 4.0)

### VADER

In [ ]:
# VADER 소셜미디어 텍스트에 최적화
# 이 영화는 정말정말 훌륭해!!!
# 훌륭하다 (기본) + 0.7 정말정말 (강조) x 1.5
# !!! (문장부호강조)  x 1.2

# 4개의 감정 지수
    # positive 긍정 확률 0 ~ 1
    # negative 부정 확률
    # neutral 중립 확률
    # compund 종합점수 -1 ~ 1
        # +1 에 가까울수록 매우 긍정적
        # -1 에 가까울수록 매우 부정적
        # 0 근처라면 중립적

# 종합점수 조회 방법 :
# score = compound_score / sqrt(compound_score**2 + 0.0625)
# score >= 0.05 긍정
# score <= -0.05 부정
# 그 사이( -0.05 < score < 0.05 )는 중립

# 대소문자 구분 AMAZING amazing 다른 점수
# :) 긍정 :-( 부정

In [3]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Playdata2\AppData\Roaming\nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

sentences = [
    "I love this product! It's absolutely amazing 😍",
    "This is the worst movie I've ever seen...",
    "The food was okay, not great but not bad either.",
    "I’m REALLY happy with the results!!!",
    "Not good at all. I’m disappointed.",
]

for s in sentences:
    scores = analyzer.polarity_scores(s)
    print(f'문장 : {s}')
    print(f'점수 : {scores}')

문장 : I love this product! It's absolutely amazing 😍
점수 : {'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.862}
문장 : This is the worst movie I've ever seen...
점수 : {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}
문장 : The food was okay, not great but not bad either.
점수 : {'neg': 0.149, 'neu': 0.487, 'pos': 0.364, 'compound': 0.4728}
문장 : I’m REALLY happy with the results!!!
점수 : {'neg': 0.0, 'neu': 0.472, 'pos': 0.528, 'compound': 0.7651}
문장 : Not good at all. I’m disappointed.
점수 : {'neg': 0.579, 'neu': 0.421, 'pos': 0.0, 'compound': -0.6711}


### 종합 정리

In [26]:
from nltk.corpus import movie_reviews
import nltk
from sklearn.metrics import accuracy_score
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

# nltk 데이터 다운로드
nltk.download('movie_reviews',quiet=True)
nltk.download('vader_lexicon',quiet=True)

# 영화 리뷰데이터 로드
fileids = movie_reviews.fileids()

In [21]:
reviews = [movie_reviews.raw(fileid) for fileid in fileids[:50]] + \
        [movie_reviews.raw(fileid) for fileid in fileids[-50:]]
categories = [movie_reviews.categories(fileid)[0]for fileid in fileids[:50]] + \
        [movie_reviews.categories(fileid)[0]for fileid in fileids[-50:]]
# + \ 이거 뭔지 ^

# +
# 리스트 합치기(concatenation) 역할

# \
# 파이썬에서 줄바꿈(line continuation) 표시
# 한 줄이 너무 길 때, 다음 줄로 이어서 쓸 수 있게 해줍니다.

len(reviews), categories.count('pos'), categories.count('neg')

(100, 50, 50)

In [24]:
# 1. TextBlob
def sentiment_textblob(docs):
    return ['pos' if TextBlob(doc).sentiment.polarity > 0 else 'neg' for doc in docs ]
predictions_textblob = sentiment_textblob(reviews)
accuracy_textblob = accuracy_score(categories,predictions_textblob)
print(f'정확도 :{accuracy_textblob:.1f}')

정확도 :0.6


In [ ]:
# 2. AFINN
def sentiment_afinn(docs):
    afn = Afinn(emoticons=True)
    return ['pos' if afn.score(doc) > 0 else 'neg' for doc in docs ]
predictions_afinn = sentiment_afinn(reviews)
accuracy_afinn = accuracy_score(categories,predictions_afinn)
print(f'정확도 :{accuracy_afinn:.1f}')

정확도 :0.7


In [30]:
# 3. Vader
def sentiment_vader(docs):
    analyzer = SentimentIntensityAnalyzer()
    return ['pos' if analyzer.polarity_scores(doc)['compound'] > 0 else 'neg' for doc in docs ]
predictions_vader = sentiment_vader(reviews)
accuracy_vader = accuracy_score(categories,predictions_vader)
print(f'정확도 :{accuracy_vader:.1f}')

정확도 :0.6


### 머신러닝 기반 감성 분석

In [36]:
# 머신러닝 기반 감성분석
from sklearn.feature_extraction.text import TfidfVectorizer # 벡터화 툴
from sklearn.naive_bayes import MultinomialNB # 모델 1
from sklearn.linear_model import LogisticRegression # 모델 2
from sklearn.model_selection import train_test_split # 데이터 분리
from sklearn.metrics import classification_report # 성능 조회

In [ ]:
# 나이브 베이즈

# 베이즈 정리
# "좋다" 단어를 본후 이 리뷰가 긍정일 확률
# p(긍정 | "좋다") = p("좋다" | 긍정) x p(긍정) / p("좋다")
    # "좋다"가 긍정일 확률 = 긍정중에 "좋다"가 있는 확률 x 전체 리뷰중 긍정 리뷰  비율 / 전체데이터에서 "좋다" 등장 비율

In [42]:
# 데이터 분할
dataset = train_test_split(reviews, categories, test_size=0.2, random_state=42, stratify=categories)
len(dataset[0]),len(dataset[2])

(80, 80)

In [45]:
# tf-idf 벡터화
vectorizer = TfidfVectorizer(max_features = 1000)
# tf-idf 벡터화
vectorizer = TfidfVectorizer(max_features = 1000)

# 학습 데이터로 fit + transform
x_train = vectorizer.fit_transform(dataset[0])

# 테스트 데이터는 transform만
x_test = vectorizer.transform(dataset[1])

# 
y_train = dataset[2]
y_test = dataset[3]

In [66]:
# MultinomialNB
mnb_clf = MultinomialNB()
mnb_clf.fit(x_train,y_train)
predict = mnb_clf.predict(x_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         neg       0.75      0.60      0.67        10
         pos       0.67      0.80      0.73        10

    accuracy                           0.70        20
   macro avg       0.71      0.70      0.70        20
weighted avg       0.71      0.70      0.70        20



In [60]:
# LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)
predict = lr.predict(x_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         neg       0.57      0.80      0.67        10
         pos       0.67      0.40      0.50        10

    accuracy                           0.60        20
   macro avg       0.62      0.60      0.58        20
weighted avg       0.62      0.60      0.58        20



### 머신러닝 기반 감성분석 성능 올리기

#### 머신러닝 감성분석 성능 향상 요소, 데이터 전처리와 벡터화 항목
 - 그외 고려항목 : 데이터 측면, 모델 선택, 하이퍼파리미터 및 학습
 https://chatgpt.com/s/t_6912a19b73688191a7195375199906c6

#### 1️⃣ 전처리(Preprocessing) 전략

1. **토큰화(Tokenization)**

   * 한국어: 형태소 단위(Okt, Mecab, Komoran 등) vs. 단어 단위
   * 영어: 단어 단위, 서브워드(BPE, WordPiece)
   * 감성 분석에서는 **의미 단위 유지**가 중요 → 형태소 단위 추천

2. **불용어 제거(Stopword Removal)**

   * ‘은’, ‘는’, ‘이’, ‘가’, ‘그리고’, ‘but’, ‘the’ 등
   * 단, 일부 감성 표현에 중요한 단어는 제외하지 않도록 주의

3. **정규화(Normalization)**

   * 대소문자 통일, 공백 제거, 특수문자/이모지 처리
   * 감성 이모지 😍, 😡 는 긍정/부정 정보가 있으므로 그대로 둘 수도 있음
   * 반복 문자 정규화: `좋아요!!!` → `좋아요!`

4. **어간 추출(Stemming) / 표제어 추출(Lemmatization)**

   * 동사/형용사 활용형 통일: ‘좋다’, ‘좋아요’, ‘좋았음’ → `좋다`
   * 감성 단어의 다양한 변형 통합 → 빈도 기반 벡터화 효과 상승

5. **n-gram 생성**

   * 단어 단위 bigram/trigram 활용 → 문맥 반영
   * 예: `not good` → 두 단어를 함께 고려하여 부정 의미 유지

6. **특수 기호·숫자 처리**

   * 감성 정보와 관련 없는 숫자 제거
   * 이모지, 강조 기호(`!!!`, `??`)는 별도로 feature로 활용 가능

---

#### 2️⃣ 벡터화(Feature) 전략

1. **기초 벡터화**

   * **Bag-of-Words (BOW)**: 단어 출현 빈도 기반
   * **TF-IDF**: 자주 등장하지만 의미 없는 단어 영향 감소

2. **n-gram 기반 Feature**

   * 단어 단위 bigram/trigram → 부정/긍정 구문 캡처
   * TF-IDF + n-gram 조합 시 성능 상승 가능

3. **Word Embedding**

   * **Word2Vec, GloVe, FastText**

     * 단어 의미 벡터 → 유사 단어 의미 반영 가능
   * **FastText**: OOV(Out-of-Vocabulary) 단어 처리 강점

4. **문장/문맥 Embedding**

   * **BERT, KoBERT, Sentence-BERT**
   * 문장 전체 의미 반영 → 단순 단어 벡터보다 감정 nuance 잡기 좋음

5. **감성 사전 기반 Feature**

   * 긍정/부정 단어 점수 합산 → TF-IDF, Word Embedding과 함께 사용 가능

6. **특징 선택/차원 축소**

   * 불필요한 단어 제거, 상관 없는 feature 줄이기
   * PCA, TruncatedSVD 등 → sparse 벡터 처리 후 성능 향상 가능

---

💡 **핵심 전략 요약**

* 전처리: 형태소 단위 토큰화 + 불용어 제거 + 정규화 + 어간 추출 + n-gram
* 벡터화: TF-IDF + n-gram OR Word Embedding(BERT 계열)
* 감성 분석에서는 **문맥 반영 + 부정 처리**가 가장 중요한 요소



In [ ]:
# 성능 향상 요소
# 소문자 변환 - 중복단어 줄이기
# 연속된 문자열 중에 3글자 이상으로 지정 - 의미없는 조사나 접미사 걸러내기
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [ ]:
# 커스텀 토크나이저 함수 만들기
def custom_tokenizer(text):
    text = text.lower()
    tokenizer =  RegexpTokenizer(r"[\w']{3,}")
    tokens = tokenizer.tokenize(text)
    porter = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    return [porter.stem(token) for token in tokens if token not in stop_words]
vector = TfidfVectorizer(
    tokenizer  = custom_tokenizer
    ,max_features=1000
    ,min_df=5
    ,max_df=0.5
    #, token_pattern = r"[\w']{3,}"
    # tokenizer를 직접 지정하면 TfidfVectorizer는 token_pattern을 무시
)
x_train = vector.fit_transform(dataset[0])
x_test = vector.transform(dataset[1])

c:\Users\Playdata2\miniconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# 모델 학습 및 성능 조회 함수
def evaluate_model(model):    
    model.fit(x_train,y_train)
    predict = model.predict(x_test)
    print( classification_report(y_test, predict))

In [70]:
evaluate_model(LogisticRegression())

              precision    recall  f1-score   support

         neg       0.80      0.80      0.80        10
         pos       0.80      0.80      0.80        10

    accuracy                           0.80        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.80      0.80      0.80        20



In [71]:
evaluate_model(MultinomialNB())

              precision    recall  f1-score   support

         neg       0.75      0.60      0.67        10
         pos       0.67      0.80      0.73        10

    accuracy                           0.70        20
   macro avg       0.71      0.70      0.70        20
weighted avg       0.71      0.70      0.70        20

